In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from string import ascii_letters
import re
import nltk
import string

In [2]:
#need to change import to updated or cleaning is called on abstracts, not raw 
import TextCleaning_updated
import importlib

In [3]:
# reading in raw data

df = pd.read_csv("../../data/original/working_federal_reporter_2020.csv")

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,7,13,14,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(df.shape)
df.head()

(1156137, 26)


,PROJECT_ID,ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,...,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008.0,1999467.0,NaN
1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,...,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008.0,1799699.0,NaN
2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,...,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008.0,1505858.0,NaN
3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,...,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008.0,51000.0,NaN
4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,...,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.074,2008.0,370996.0,NaN


In [5]:
df = df.rename(columns = {"ABSTRACT": "RAW_ABSTRACT"})

In [6]:
df.head()

,PROJECT_ID,RAW_ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,...,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008.0,1999467.0,NaN
1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,...,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008.0,1799699.0,NaN
2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,...,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008.0,1505858.0,NaN
3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,...,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008.0,51000.0,NaN
4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,...,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.074,2008.0,370996.0,NaN


In [7]:
#Updated because original references wrong file
importlib.reload(TextCleaning_updated)
df = TextCleaning_updated.remove_nulls(df,"RAW_ABSTRACT")
df = TextCleaning_updated.remove_duplicates(df)

42380 nulls in  RAW_ABSTRACT . These rows removed.
79187 duplicate abstracts removed
0 project ID duplicates - not removed


In [8]:
# lowercase all abstracts, strip leading and trailing whitespace,
# and save in a working abstract column that will be updated as text is cleaned

df["working_abstract"] = [abstract.lower().strip() for abstract in df["RAW_ABSTRACT"]] 
df = TextCleaning_updated.drop_empties(df)


In [9]:
df.head()

,PROJECT_ID,RAW_ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,...,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,working_abstract
0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,...,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008.0,1999467.0,NaN,"this is a project to explore game-based, metap..."
1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,...,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008.0,1799699.0,NaN,institution: franklin institute science museum...
2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,...,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008.0,1505858.0,NaN,through programs (including small group conver...
3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,...,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008.0,51000.0,NaN,in partnership with the american chemical soci...
4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,...,NY,14850-2820,UNITED STATES,NaN,NaN,47.074,2008.0,370996.0,NaN,amphibian populations around the world are exp...


In [10]:
importlib.reload(TextCleaning_updated)
df = TextCleaning_updated.remove_short_abstracts(df,limit=150)

2205 short abstracts removed


In [11]:
# Start phrases to remove

start_phrases=['****technical abstract****',
               '****non technical abstract****',
               '*** non- technical abstract ***',
               '**non-technical abstract**',         
               '*non-technical abstract*',
               '*****non-technical abstract*****',
               '***** non-technical abstract *****',
               '****non-technical abstract****',
               '***non-technical abstract***',
               '****nontechnical abstract****',
               'technical summary', 
               'nontechnical summary',
               'non-technical summary',
               'non-technical description',
               'description (provided by the applicant)',
               'description (provided by investigator)',  
               'description (provided by applicant)',
               'Description (provided by applicant)',
               'project summary/abstract',   
               'abstract',
               'proposal abstract',
               'research abstract',
               'project summary',
               'PROJECT SUMMARY'
               'summary',
               'research summary',
               'proposal',
               'description',
               'project description'
               'narrative',
               '(see instructions):',
               '\t',
               'for center application (provided by the investigator):',
               'objective(s)',      
               'exceed the space provided',
               'provided by applicant',
               'provided by candidate',
               'one page and must contain a summary of the proposed activity suitable for dissemination to thepublic. it should be a self-contained description of the project and should contain a statement of objectives and methods to be employed.it should be informative to other persons working in the same or related fields and insofar as possible understandable to a technically liter-ate lay reader. this abstract must not include any proprietary/confidential information.* please click the add attachment button to complete this entry.',
                #Added new ones
                'introduction','this proposal','the proposal','this is a project','this', "in","the",'project']
            
wa = 'working_abstract'

df[wa]=df[wa].apply(str.lstrip,args=[' ?-_^. :,!;¿|()[]]#%>﻿&\''])

#Remove found phrases
for phrase in start_phrases:
    df[wa]=df[wa].apply(TextCleaning_updated.remove_phrase,args=[phrase,'Start']).apply(str.lstrip,args=[' :./)'])
#Repeated in case the order of project summary/abstract varies
for phrase in start_phrases:
    df[wa]=df[wa].apply(TextCleaning_updated.remove_phrase,args=[phrase,'Start']).apply(str.lstrip,args=[' :./)'])

# Often, sentences will start with - or *, 
# but they indicate other quality issues and don't end with them,so it's okay to remove them
df[wa]=df[wa].apply(str.lstrip,args=['?-*_^. :,!;=¿|]#%>&-\t\n'])   
    
df = TextCleaning_updated.drop_empties(df)
    
# update Start Char column in df
df['Start Char']=df[wa].apply(lambda x: x[0])




In [12]:
df.head(200)

,PROJECT_ID,RAW_ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,...,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,working_abstract,nchar,Start Char
0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,...,UNITED STATES,NaN,NaN,47.076,2008.0,1999467.0,NaN,"to explore game-based, metaphor enhanced (game...",2057,t
1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,...,UNITED STATES,NaN,NaN,47.076,2008.0,1799699.0,NaN,stitution: franklin institute science museum p...,2053,s
2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,...,UNITED STATES,NaN,NaN,47.076,2008.0,1505858.0,NaN,through programs (including small group conver...,1154,t
3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,...,UNITED STATES,NaN,NaN,47.049,2008.0,51000.0,NaN,partnership with the american chemical society...,875,p
4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,...,UNITED STATES,NaN,NaN,47.074,2008.0,370996.0,NaN,amphibian populations around the world are exp...,1322,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,89880.0,SIMULATION OF SMOKE FROM THE SIBERIAN FOREST F...,2008.0,Fire - disasters; forest; Measurement; North A...,SIMULATION OF SMOKE FROM THE SIBERIAN FOREST F...,NASA,NASA,NaN,NNX08BA94G,9/17/2008,...,UNITED STATES,NaN,NaN,43.AAA,2008.0,36741.0,NaN,simulation of smoke from the siberian forest f...,255,s
213,89879.0,This proposal will demonstrate a software infr...,2008.0,Blood Vessels; Brain; cluster computing; Code;...,MULTISCALE MODELS AND PETAFLOPS SIMULATIONS ON...,NSF,NSF,NaN,0845449,10/1/2008,...,UNITED STATES,NaN,NaN,47.080,2008.0,149999.0,NaN,will demonstrate a software infrastructure spe...,596,w
214,89878.0,"PI: Vesel, Judy Institution: TERC Inc Title: S...",2008.0,abstracting; base; Biological Sciences; Calcul...,SHARED SIGNING SCIENCE PLANNING PROJECT,NSF,NSF,NaN,0754587,10/1/2008,...,UNITED STATES,NaN,NaN,47.076,2008.0,74893.0,NaN,"pi: vesel, judy institution: terc inc title: s...",3231,p
215,89877.0,Classroom students need to engage with innovat...,2008.0,Address; base; career; Characteristics; Collab...,ITEST STUDY: PROMOTING STEM CAREER INTEREST IN...,NSF,NSF,NaN,0833524,10/1/2008,...,UNITED STATES,NaN,NaN,47.076,2008.0,1129730.0,NaN,classroom students need to engage with innovat...,2114,c


In [13]:
# remove junk at end
#Made all lowercase
end_phrases = ['(end of abstract)',
               'end of abstract', 
               '(abstract end)',  
               '(end of abstract.)',
               '(end 0f abstract)',
               '(end of absract)',
               '(abstract below)',
               '(end of reviewers\' comment)',
               '(end abstract)',
               
               'performance site ========================================section end===========================================',
               'key personnel ========================================section end===========================================',
               '[summary truncated at 7800 characters]', 
               'this award reflects nsf\'s statutory mission and has been deemed worthy of support through evaluation using the foundation\'s intellectual merit and broader impacts review criteria.',
               'project description page 6', 'page 1 of 1', 'project summary/abstract page 6',
               'project description page 7', 'project summary/abstract page 7', 'pag 1 o 1', 
               'page 2 number pages consecutively at the bottom throughout form page 2']

df = TextCleaning_updated.remove_junk_end(df, 'working_abstract', end_phrases) 

(end of abstract)
end of abstract
(abstract end)
(end of abstract.)
(end 0f abstract)
(end of absract)
(abstract below)
(end of reviewers' comment)
(end abstract)
performance site ========================================section end===========================================
key personnel ========================================section end===========================================
[summary truncated at 7800 characters]
this award reflects nsf's statutory mission and has been deemed worthy of support through evaluation using the foundation's intellectual merit and broader impacts review criteria.
project description page 6
page 1 of 1
project summary/abstract page 6
project description page 7
project summary/abstract page 7
pag 1 o 1
page 2 number pages consecutively at the bottom throughout form page 2


In [14]:
punctuation=['!','?','.']

#y='*' #Generally if last char is '*', it comes in as '***' following a complete sentence, so unlikely to be 
# cut-of'

#Fixes '***' if that makes the last character a punctuation ending mark, otherwise should remove
y='*' 
entries_ending_right=df.loc[df['LAST_CHAR']==y]

entries_ending_right['new_last_char_possible']= entries_ending_right.apply(lambda x:TextCleaning_updated.remove_phrase(x[wa],'***',loc='End')[-1],axis=1)
entries_to_fix=list(entries_ending_right[entries_ending_right['new_last_char_possible'].isin(punctuation)]['PROJECT_ID'])


df[wa]=df.apply(lambda x: TextCleaning_updated.remove_phrase(x[wa],'***','End') 
                if x['PROJECT_ID'] in entries_to_fix else x[wa],axis=1)

df['LAST_CHAR']=df[wa].apply(lambda x: x[-1])


/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [15]:
# Using regular expressions to remove "junk" within text body - not necessarily at the start or end

#'Enter the text here tha' ending with 'lines of text.'
expression=re.compile('Enter the text here that.*lines of text')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))

expression=re.compile('PHS .*?Continuation Format Page')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))
expression=re.compile('OMB No .*?Continuation Format Page')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))



df[wa]=df[wa].replace('Project Summary/Abstract','')

"""If it starts with 'one page and must contain',
#This is an NIH thing and there aren't that many of them, but come from 3 different cfda
it will start with "one page and must contain a summary of the proposed activity suitable for dissemination to thepublic. 
It should be a self-contained description of the project and should contain a statement of objectives and methods to be employed.
It should be informative to other persons working in the same or related fields and insofar as possible understandable to a technically liter-ate lay reader. 
This Abstract must not include any proprietary/confidential information.* 
Please click the add attachment button to complete this entry." plus some attachments, which includes tracking number, twice:
following the second trackign number, there is a grant number followed by the actual content" 

At the end of these files, they all end in 'Project Narrative File'(last instance) followed by more attachments, all of which can be discarded
"""
expression1=re.compile('one page and must.*?Tracking Number.*?(Tracking Number)')
expression2=re.compile('Project Narrative File.*')
def fix_abstract(abstract):
    if abstract.startswith('one page and must contain'):
        abstract=re.sub(expression1,'',abstract)
        return re.sub(expression2,'',abstract)
    else:
        return abstract

df[wa]=df[wa].apply(fix_abstract)


df[wa]=df[wa].apply(lambda x: x.lstrip(',;\n\t&-!'))
df=df.loc[df[wa].apply(len)>0]


In [16]:
#starting_exact_phrases to remove
#'This subproject represents an estimate of the percentage of the CTSA funding that isbeing utilized for a broad area of research (AIDS research, pediatric research, orclinical trials).  The Total Cost listed is only an estimate of the amount of CTSAinfrastructure going towards this area of research, not direct funding provided bythe NCRR grant to the subproject or subproject staff.'
#'This subproject is one of many research subprojects utilizing theresources provided by a Center grant funded by NIH/NCRR. The subproject andinvestigator (PI) may have received primary funding from another NIH source,and thus could be represented in other CRISP entries. The institution listed isfor the Center, which is not necessarily the institution for the investigator.'
df[wa]=df[wa].apply(lambda x: x.replace('This subproject represents an estimate of the percentage of the CTSA funding that isbeing utilized for a broad area of research (AIDS research, pediatric research, orclinical trials).  The Total Cost listed is only an estimate of the amount of CTSAinfrastructure going towards this area of research, not direct funding provided bythe NCRR grant to the subproject or subproject staff.',
                                       ''))

expression=re.compile('This subproject is one of many research subprojects.*not necessarily the institution for the investigator.')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))
expression=re.compile('This subproject is one of many research subprojects.*to the subproject or subproject staff.')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))
  

In [17]:
def remove_long_phrase(record):
    """ ignores case to remove multi-word phrases in a particular order, especially those likely to run into other words,
    e.g. Institution university of washingtonPI mary williams. This doesn't work when titles or insititutions have escape characters in them, which is a bummer
    see for example ENHANCING THE USE OF NASA EARTH SCIENCE RESULTS / DATA / AND TECHNOLOGY BY ENGAGING THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS COMMUNITIES OF
    PRACTICE IN TARGET AREAS OF INTEREST TO NASA THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS (''FED"""
    title=record['PROJECT_TITLE']
    try:
        new_abstract=re.sub(title,'',record[wa],flags=re.IGNORECASE)      
        return re.sub(record['ORGANIZATION_NAME'],'',new_abstract,flags=re.IGNORECASE)   
    except:
        try:
            return re.sub(record['ORGANIZATION_NAME'],'',record[wa],flags=re.IGNORECASE)   
        except:
            return record[wa]

In [18]:
df[wa]=df.apply(lambda x: remove_long_phrase(x),axis=1)

expression=re.compile('Project Summary/Abstract Page.*')

def remove_contact_pd(x):
    """removes clause at end that tends to occur: eg Project Summary/Abstract Page 222Contact PD/PI: Sampson, HughNarrative ("""
    if x.startswith('Contact PD/PI'):
        return re.sub(expression,'',x)
    else:
        return x
df[wa]=df[wa].apply(remove_contact_pd) 

In [19]:
#Drop those that are now length 0 (ie were all punctuation or removable phrases
df.drop(list(df[df[wa].apply(lambda x: len(x)==0)].index),inplace=True)
df['Start Char']=df[wa].apply(lambda x:x[0])
df['LAST_CHAR']=df[wa].apply(lambda x:x[-1])
df['nchar']=df[wa].apply(lambda x: len(x))
#df.drop(df[df[wa].apply(len)==0].index[0],axis=0,inplace=True)

In [20]:

#####################
#Additional expressions we could choose to remove
#Identify abstracts with excessive amounts of other fields to uncover additional bad abstract types
#If we wanted to be on the safe side, some EDA makes me think we could remove anything with more than 3 or 4 of these fields. It's where they start getting wonky.
###################

fields=['Principal Investigator','Program Director','Attachment','Instructions','Lines',
        'Space Provided','Performance Site','Organization','Key Personnel']
all_fields=fields.copy()
all_fields.extend([x.lower() for x in fields])
all_fields.extend([x.upper() for x in fields])
all_fields.extend(['PI','Form','Page','Title','.pdf','.doc'])

def count_up_fields(abstract):
    count=0
    for field in all_fields:
        if field in abstract:
            count+=1
    return count

df['Field Count']=df[wa].apply(count_up_fields)


In [21]:
df.to_pickle("./clean_dataset.pkl")
df.to_csv('./FRAbstractsSqueakyClean.csv')


In [22]:
###########################
#Additional expressions we could remove, but there is a small possibility of some information being lost
##########################

#Issues: 'Close FormNextPrint PageAbout OMB Number']#This is usually ended with "Project summary", 
#so anything between those 2 can be delete, and #ended with a clause starting with 'Close FormProject' and ending in'Narrative File'

#expression1=re.compile('Close FormNext.*?Project Summary')
#expression2=re.compile('Close FormProject.*Narrative File')
def fix_abstract(abstract):
    if abstract.startswith('Close FormNext'):
        abstract=re.sub(expression1,'',abstract)
        return re.sub(expression2,'',abstract)
    else:
        return abstract
df[wa]=df[wa].apply(fix_abstract)

#If ends in 'Description,', then go to last instance of PERFORMANCE (for Performance SITES), otherwise "KEY PERSONNEL", upper case, and cut all that follows

#expression1=re.compile('PERFORMANCE.*Description,$')
#expression2=re.compile('KEY PERSONNEL.*Description,$')

def apply_expressions(abstract):
    if abstract.endswith('Description,'):
        if re.search(expression1,abstract) != None:
            return re.sub(expression1,'',abstract)
        else:
            return re.sub(expression2,'',abstract)
    else:
        return abstract
    
#df[wa]=df[wa].apply(apply_expressions)

#expression1=re.compile('PERFORMANCE.*Page 3$')
#expression2=re.compile('KEY PERSONNEL.*Page 3,$')

def apply_expressions(abstract):
    if abstract.endswith('Description,'):
        if re.search(expression1,abstract) != None:
            return re.sub(expression1,'',abstract)
        else:
            return re.sub(expression2,'',abstract)
    else:
        return abstract
    
#df[wa]=df[wa].apply(apply_expressions)



In [23]:
df.head()

,PROJECT_ID,RAW_ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,...,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,working_abstract,nchar,Start Char,LAST_CHAR,Field Count
0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,...,NaN,47.076,2008.0,1999467.0,NaN,"to explore game-based, metaphor enhanced (game...",2039,t,.,0
1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,...,NaN,47.076,2008.0,1799699.0,NaN,"stitution: science museum pi: snyder, steve p...",1911,s,.,0
2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,...,NaN,47.076,2008.0,1505858.0,NaN,through programs (including small group conver...,1154,t,.,0
3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,...,NaN,47.049,2008.0,51000.0,NaN,partnership with the american chemical society...,823,p,.,0
4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,...,NaN,47.074,2008.0,370996.0,NaN,amphibian populations around the world are exp...,1322,a,.,1


In [24]:
#Do we still have duplicate abstracts?
#df[wa].value_counts()


In [25]:
df[wa]

0          to explore game-based, metaphor enhanced (game...
1          stitution:  science museum pi: snyder, steve p...
2          through programs (including small group conver...
3          partnership with the american chemical society...
4          amphibian populations around the world are exp...
                                 ...                        
1121097    objective 1: determine how diet, the interacti...
1121098    objective 1:  determine the accumulation and v...
1121099    1. evaluate plant through micro-patch scale re...
1121100    objective 1:  demonstrate effects of parental ...
1121101    objective 1: develop microbial and enzymic app...
Name: working_abstract, Length: 1028720, dtype: object

In [26]:
#def split_join_words(c):
    #cleaned = "".join(re.findall('[A-Z][^A-Z]*',c))
    #return cleaned
#df[wa] = df[wa].apply(split_join_words)




In [27]:
smalldf = df[:100]

In [28]:
#df['text_clean2'] = [tuple(x) for x in df['working_abstract']]
#finder = BigramCollocationFinder.from_words(df["text_clean2"])
#finder.apply_freq_filter(3)
# return the 10 n-grams with the highest PMI
#print(finder.nbest(bigram_measures.pmi, 1))

In [29]:
#import nltk
#from nltk.collocations import *
#Identifying Junk Phrases 
#nltk.download("genesis")

#bigram_measures = nltk.collocations.BigramAssocMeasures()
#trigram_measures = nltk.collocations.TrigramAssocMeasures()

# change this to read in your data
#finder = BigramCollocationFinder.from_words(
    #nltk.corpus.genesis.words(smalldf[wa]))

# only bigrams that appear 3+ times
#finder.apply_freq_filter(3)

# return the 10 n-grams with the highest PMI
#finder.nbest(bigram_measures.pmi, 10)

In [30]:
df.head()

,PROJECT_ID,RAW_ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,...,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,working_abstract,nchar,Start Char,LAST_CHAR,Field Count
0,89996.0,"This is a project to explore Game-based, Metap...",2008.0,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,...,NaN,47.076,2008.0,1999467.0,NaN,"to explore game-based, metaphor enhanced (game...",2039,t,.,0
1,89997.0,Institution: Franklin Institute Science Museum...,2008.0,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,...,NaN,47.076,2008.0,1799699.0,NaN,"stitution: science museum pi: snyder, steve p...",1911,s,.,0
2,89998.0,Through programs (including small group conver...,2008.0,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,...,NaN,47.076,2008.0,1505858.0,NaN,through programs (including small group conver...,1154,t,.,0
3,89999.0,In partnership with the American Chemical Soci...,2008.0,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,...,NaN,47.049,2008.0,51000.0,NaN,partnership with the american chemical society...,823,p,.,0
4,90000.0,Amphibian populations around the world are exp...,2008.0,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,...,NaN,47.074,2008.0,370996.0,NaN,amphibian populations around the world are exp...,1322,a,.,1
